In [1]:
import os
import sys
import pandas as pd
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [2]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(1)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
end_page
contents_num = 7

blog_title_lst = []
blog_url_lst = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
    
    # 블로그 제목, url 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_title_lst.append(title)
        blog_url_lst.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_title_lst)
print(blog_url_lst)

2024-05-03 2024-05-10
총 블로그글 개수 :  1,065건
연관검색어: 
구로디지털단지역 술집구로디지털단지역 카페구로디지털단지역 미용실구로디지털단지역 피부과구로디지털단지역 오피스텔구로디지털단지역 네일구로디지털단지역 아파트구로디지털단지역 별밤구로디지털단지역 공영주차장구로디지털단지역 다이소구로디지털단지역 주차장구로디지털단지역 꽃집구로디지털단지역 놀거리구로디지털단지역 스타벅스구로디지털단지역 호텔구로디지털단지역 영화관구로디지털단지역 빵집구로디지털단지역 원룸구로디지털단지역 브런치구로디지털단지역 이마트


['구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]', '구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료', '구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점', '구로디지털단지역 맛집 흐뭇한 데이트 기대되는 양고기 양원집', '구로디지털단지역 맛집 가성비 소고기 세트가 있는 조선화로집 구로디지털점', '도체오타워 임대 구로동 구로디지털단지역 사무실', '지식산업센터 임대 구로디지털단지역 역세권 사무실 대륭포스트타워3차', '구로디지털단지역안경 으뜸50안경 구로디지털단지역점 여행대비 선글라스', '[구로디지털단지역 맛집] 특색있는 프리미엄 양갈비 맛집, 양원집 구로디지털단지점!', '구로디지털단지역 PT 여성전문헬스장 휘트니스피플우먼 구로디지털단지점 PT 체험...', '구로디지털단지역피부과 얼굴 선이 갸름하게', '구로디지털단지역치킨 정석닭강정 구로점 단체주문 가능 맛집', '구로디지털단지역 반응 좋은 조개찜 맛집 물찬조개', '구디고깃집 / 형제특수부위 고기가 맛있는 구로디지털단지역 맛집', '구로디지털단지역 생활권 관악구 신림동 신축빌라 분양 매매', '구로디지털단지역 맛집 돼지왕갈비맛집 조선화로집구로디지털점', '[구로디지털단지역 맛집] 경성양꼬치', '서울 구로디지털단지역 근처 롯데시티호텔 구로 1박 후기', '구디 브레드밀레 연유빵 먹으러 구로디지털단지역 빵집까지 다녀온 후기', '[계문산장] 

In [3]:
# 확인
print(len(blog_title_lst),len(blog_url_lst))

1065 1065


In [4]:
df = pd.DataFrame({"title":blog_title_lst, "url":blog_url_lst})
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1065 non-null   object
 1   url     1065 non-null   object
dtypes: object(2)
memory usage: 16.8+ KB


,title,url
0,구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향],https://blog.naver.com/agness888/223439262998
1,구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료,https://blog.naver.com/lavish001/223440020562
2,구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점,https://blog.naver.com/jang9369/223439007190
3,구로디지털단지역 맛집 흐뭇한 데이트 기대되는 양고기 양원집,https://blog.naver.com/jej3480/223440004458
4,구로디지털단지역 맛집 가성비 소고기 세트가 있는 조선화로집 구로디지털점,https://blog.naver.com/v8v8v8v8v8/223433538129
...,...,...
1060,목구멍 구로디지털단지점 구디맛집 미나리도 같이 나오네요.,https://blog.naver.com/hwsa5599/223434987566
1061,참숯화로구이가 맛있는 구로동맛집 미켈고깃집,https://blog.naver.com/dicorhkdlf/223435511013
1062,구로디지털단지맛집 급냉눈꽃 목살 맛있는 서울반달집 별관,https://blog.naver.com/nejung-/223436305762
1063,구로디지털단지맛집 신선하고 새로운 맛에 반하나? 얼큰하게 부대찌개대사관,https://blog.naver.com/pinky4409/223435237341


- blog 본문 가져오기(test)

In [5]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# 해당 blog 불러오기 
url = blog_url_lst[0]
driver.get(url)
time.sleep(1)


# iframe 접근 
driver.switch_to.frame('mainFrame')


# 제목 가져오기
overlays_title = ".se-module.se-module-text.se-title-text"
html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
title = html_title.text
print(title)


# 포스팅 날짜 가져오기
overlays_publishDate = ".se_publishDate.pcol2"
html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
date = html_publishDate.text
print(date)

# 본문 가져오기 
overlays_content = ".se-component.se-text.se-l-default"
html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
contents_lst = []
for content in html_content : 
    contents_lst.append(content.text.replace('\n',''))
    

print(contents_lst)

# 해시태그 가져오기 

# title = html_title.text 
# title





구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]
2024. 5. 7. 19:13
['누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요.부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요!다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요!', '저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요.평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요.', '그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요', '메뉴도 프리미엄 등급 토마호크와한돈 꽃목살, 한돈 숄더랙, 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요.', '메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요!결론적으로는 잘했다고 봅니다.쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요.그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요.이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다.', '', '젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요!식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요. 이날도마찬가지였어요. 쌈을 싸먹을 때도잘 어울린답니다.약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요.', '고기 좀 좋아한다, 먹을 줄 안다~이런 분이면 아실 거에요.숯이 얼마나 중요한지를 말이에요.좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠.그래서 항상 어떤 숯이 

- blog 본문 크롤링

In [9]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome()

crawling_blog_data = []


# blog 본문 가져오기 
for i in range(10) :    
    url = str(blog_url_lst[i])
    driver.get(url)
    time.sleep(1)
    
    
    # 본문 크롤링 예외 처리 
    try :
        # iframe 접근 
        driver.switch_to.frame('mainFrame')

        
        # 제목 가져오기
        overlays_title = ".se-module.se-module-text.se-title-text"
        html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
        title = html_title.text

        
        # 작성 날짜 가져오기
        overlays_publishDate = ".se_publishDate.pcol2"
        html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
        date = html_publishDate.text

        
        # 본문 가져오기
        overlays_content = ".se-component.se-text.se-l-default"
        html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
        contents_lst = []
        for content in html_content : 
            contents_lst.append(content.text.replace('\n',''))
        content_str = ''.join(contents_lst)

        
        # 크롤링한 내용 담기 
        crawling_blog_data.append({"title":title, "date":date, "content":content_str})
        
        # driver 종료 
        driver.close
        time.sleep(1)
        
        
    # 에러 발생시 다음 블로그로 이동
    except : 
        print("error",i,title)
        driver.close()
        time.sleep(1)


In [10]:
print(crawling_blog_data)

[{'title': '구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향]', 'date': '2024. 5. 7. 19:13', 'content': '누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘가에가야 하는 경우 더 그렇지 않나 싶어요.부모님과 친한 편인데도 불구하고어버이날 무엇을 준비하면 좋을지쉽사리 결정 내리지 못했어요!다행히 이번엔 구로디지털단지역 맛집에서제 주도 하에 가족이 모여 식사를 했는데맛도 그렇고 친절함 등 전반적으로만족스러워서 뿌듯한 미리 어버이날기념으로 먹고왔어요!저희와 같은 생각인지 많이들 외식을하는 날이라 저는 미리 예약을 했는데신의 한 수였어요.평이 좋은 여느 음식점이 보통 그런데여기도 식사 시간대에는 손님이꽤 몰리는 모습이었거든요.그래도 복잡한 분위기는 아니라부모님 모시고 가기에 무리는없었어요메뉴도 프리미엄 등급 토마호크와한돈 꽃목살, 한돈 숄더랙, 그리고우대갈비까지 아무 곳에서나맛볼 수 없는 귀한 부위 위주로준비되어 있으니 특별한 날에방문하기 좋은 곳이었어요.메뉴는 부모님이 드시고 싶어하시는걸로 선택을 해서 주문했어요!결론적으로는 잘했다고 봅니다.쉽게 접하기 어려운 부위이기도 하고맛도 굉장히 훌륭했으니까요.그렇게 주문하고 기다리다 보니기본 반찬이 테이블 위로하나 둘 올라오기 시작했어요.이건 무를 적당히 얇게 썰어서양념한 건데 고기를 먹다 보면 어느새슬쩍 스며드는 느끼함을 잡아줄 때큰 역할을 해주었습니다.젓갈은 밥이랑 먹을 때도 맛있는데의외로 고기랑도 잘 어울리더라고요!식당에서 식사할 때마다 젓갈을 주면그렇게 반가운데요. 이날도마찬가지였어요. 쌈을 싸먹을 때도잘 어울린답니다.약간 흐물거리는 게 나오는 곳도있는데 구로디지털단지역 맛집은기본찬 수준도 훌륭해서 쫄깃하고감칠맛까지 났어요.고기 좀 좋아한다, 먹을 줄 안다~이런 분이면 아실 거에요.숯이 얼마나 중요한지를 말이에요.좋지 않은 걸 쓰면 일단 건강에도좋지 않고 향이 베이지 않아 불맛을입히는 데에도 지장이 생기는불상사가 생기죠.그래서 항상

In [13]:
df=pd.DataFrame(crawling_blog_data)
df.index = range(1,11)
df

,title,date,content
1,구로디지털단지역 맛집 프리미엄 부위가 일품 [참숯구이 육향],2024. 5. 7. 19:13,누구라도 특별한 날이 다가오면뭐 할지 고민할 것 같은데요.특히나 어른을 모시고 어딘...
2,구로디지털단지역한의원 또는 양천구병원에서 입원 재활치료,2024. 5. 8. 12:42,"평소 매일 이용하는 도로라고 해도 불쑥 튀어나올 수 있는 모든 차량, 보행자를 주의..."
3,구로디지털단지역 고기집 야자숯에 구워 풍부한 육즙이 느껴진 인생맛껍 구로본점,2024. 5. 7. 15:38,위치 : 서울 구로구 디지털로34길 55 B107호영업시간 : 12:00 ~ 23:...
4,구로디지털단지역 맛집 흐뭇한 데이트 기대되는 양고기 양원집,2024. 5. 8. 21:50,구로디지털단지역 맛집 흐뭇한 데이트 기대되는 양고기 양원집📌 서울 구로구 디지털로3...
5,구로디지털단지역 맛집 가성비 소고기 세트가 있는 조선화로집 구로디지털점,2024. 5. 4. 9:30,안녕하세요 러블리뷰입니다🍭친구랑 오랜만에 장에 기름칠을 하러 구디에 왔어요구디에 맛...
6,도체오타워 임대 구로동 구로디지털단지역 사무실,2024. 5. 8. 19:52,도체오타워구로디지털단지역 사무실서울사무실 _ 구로구 시흥대로 구로디지털단지역 역세권...
7,지식산업센터 임대 구로디지털단지역 역세권 사무실 대륭포스트타워3차,2024. 5. 8. 15:00,안녕하세요!알스퀘어는 가산·구로 130개 지식산업센터를 직접 전수조사하여조건에 딱 ...
8,구로디지털단지역안경 으뜸50안경 구로디지털단지역점 여행대비 선글라스,2024. 5. 7. 17:04,여름의 필수품 하면 선글라스이지요.강렬한 햇빛과 자외선으로부터 눈을 보호하기 위해 ...
9,"[구로디지털단지역 맛집] 특색있는 프리미엄 양갈비 맛집, 양원집 구로디지털단지점!",2024. 5. 7. 1:55,오랜만에 구로디지털단지역 맛집!그것도 엄청 맛있는 양고기 집이라구요? ㅎㅎ대로변쪽에...
10,구로디지털단지역 PT 여성전문헬스장 휘트니스피플우먼 구로디지털단지점 PT 체험 후기...,2024. 5. 8. 13:59,"산후에 찐 살들을 빼기위해헬스, 필라테스 등 다양한 운동을 경험하고 있어요!개인적으..."
